# Vector Search with Cloud SQL

In this notebook, we'll leverage the Vector Search capabilities available in [InterSystems IRIS Cloud SQL](https://developer.intersystems.com/products/iris-cloud-sql-integratedml/). The feature works in exactly the same way as in the InterSystems IRIS 2024.1 release, but Cloud SQL requires secure connections, and this notebook illustrates how to set those up.

First, please adapt the password and hostname entries in the following cell to match your Cloud SQL deployment.

In [ ]:
username = 'SQLAdmin'
password = '...'
hostname = '...'
port = 443 
namespace = 'USER'

## Setting up the SSL configuration

SSL configurations for DB-API connections are based on the same infrastructure as ODBC and .NET. That is, they'll look for configuration settings in a `.ini` file specified through the `ISC_SSLconfigurations` environment variable. In the demo container, this environment variable points to `/usr/cert-demo/SSLConfigs.ini`, which includes all the settings required for establishing a secure connection:

    ```
    [CloudSQL] 
    CertFile=/usr/cert-demo/certificateSQLaaS.pem 
    KeyType=2 
    Protocols=28 
    CipherList=ALL:!aNULL:!eNULL:!EXP:!SSLv2 
    VerifyPeer=0 
    VerifyDepth=9 
    ```

The following code snippet is somewhat redundant, but if you're running this notebook somewhere else it may come in handy to set up the SSL configuration file for you.

In [ ]:
import os

# The SSL config name we'll use when connecting
sslConfigName = "CloudSQL"

sslConfigFile = os.getenv('ISC_SSLconfigurations')
if (sslConfigFile == None):
    print("ERROR: ISC_SSLconfigurations is not set!")

# if SSL config file does not exist, create it
if (os.path.exists(sslConfigFile)):
    print("Found existing SSL config file, assume it's intentional")
else:
    certificateFile = (os.path.dirname(sslConfigFile) + "/certificateSQLaaS.pem")
    ini = "[%s] \n" % sslConfigName
    ini += "CertFile=%s \n" % certificateFile
    ini += "KeyType=2 \n"
    ini += "Protocols=28 \n"
    ini += "CipherList=ALL:!aNULL:!eNULL:!EXP:!SSLv2 \n"
    ini += "VerifyPeer=0 \n"
    ini += "VerifyDepth=9 \n"
    with open(sslConfigFile, 'wt') as f:
        f.write(ini)
    print("Wrote new SSLConfigs.ini file")

    if (os.path.exists(certificateFile) == False):
        print("Please make sure your certificate file is located at '%s' before proceeding" % certificateFile)

### Copying the certificate

Now that the SSL configurations are in place, make sure the `certificateSQLaaS.pem` file for your Cloud SQL deployment is located at the path described in the previous cell's output (defaults to `/usr/cert-demo/`). You can download the certificate file from your deployment's detail screen. Look for the button that says "Get X.509 certificate".

Next, you'll have to copy it into the container. Adapt the download location of the certificate and the name of the target container in following command to suit your environment.

```Shell
docker cp ~/Downloads/certificateSQLaas.pem iris-vector-search-jupyter-1:/usr/cert-demo/
```

We'll use [simple DB-API commands](https://docs.intersystems.com/irislatest/csp/docbook/DocBook.UI.Page.cls?KEY=BPYNAT_pyapi#BPYNAT_pyapi_impl_conn) in this example:

In [ ]:
import iris

connection = iris.connect( hostname, port, namespace, username, password, sslconfig = "CloudSQL" )
cursor = connection.cursor()

cursor.execute("SELECT 'hello secure world!'")
cursor.fetchone()[0]

## Double vectors, no ice

Now that we have established a secure connection, let's get onto some actual vector stuff!

See the neighbouring `sql_demo.ipynb` for full detail on what we're trying to achieve here.

In [ ]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('../data/scotch_review.csv')
df.head()

In [ ]:
# Clean data
# Remove the specified columns
df.drop(['currency'], axis=1, inplace=True)

# Drop the first column
df.drop(columns=df.columns[0], inplace=True)

# Remove rows without a price
df.dropna(subset=['price'], inplace=True)

# Ensure values in 'price' are numbers
df = df[pd.to_numeric(df['price'], errors='coerce').notna()]

# Replace NaN values in other columns with an empty string
df.fillna('', inplace=True)

df.head()

In [ ]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained sentence transformer model. This model's output vectors are of size 384
model = SentenceTransformer('all-MiniLM-L6-v2') 

# Generate embeddings for all descriptions at once. Batch processing makes it faster
embeddings = model.encode(df['description'].tolist(), normalize_embeddings=True)

# Add the embeddings to the DataFrame
df['description_vector'] = embeddings.tolist()

df.head()

## And now load them into Cloud SQL

We'll first create a table and then ingest all the rows from the dataframe we created earlier.

In [ ]:
cursor.execute('DROP TABLE IF EXISTS scotch_reviews')
cursor.execute(f"""CREATE TABLE scotch_reviews (
                    name VARCHAR(255),
                    category VARCHAR(255),
                    review_point INT,
                    price DOUBLE,
                    description VARCHAR(2000),
                    description_vector VECTOR(DOUBLE, 384)
                )""")

seq = []
for index, row in df.iterrows():
    seq.append((row['name'], row['category'], row['review.point'], row['price'], row['description'], str(row['description_vector'])))

success = cursor.executemany("INSERT INTO scotch_reviews (name, category, review_point, price, description, description_vector) VALUES (?, ?, ?, ?, ?, TO_VECTOR(?))", seq)


In [ ]:
description_search = "earthy and creamy taste"
search_vector = model.encode(description_search, normalize_embeddings=True).tolist() # Convert search phrase into a vector

cursor.execute("""
            SELECT TOP 3 * FROM scotch_reviews 
            WHERE price < 100 
            ORDER BY VECTOR_DOT_PRODUCT(description_vector, TO_VECTOR(?)) DESC
        """, [str(search_vector)])

print(cursor.fetchall())

## Langchain

Light adaptation of `langchain_demo.ipynb`

In [ ]:
from langchain.docstore.document import Document
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

from langchain_iris import IRISVector


In [ ]:
loader = TextLoader("../data/state_of_the_union.txt", encoding='utf-8')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=400, chunk_overlap=20)
docs = text_splitter.split_documents(documents)

Establish a secure connection using SQLAlchemy

In [ ]:
from sqlalchemy import create_engine, text
import ssl

url = f"iris://{username}:{password}@{hostname}:{port}/{namespace}"
sslcontext = ssl.create_default_context(cafile="/usr/cert-demo/certificateSQLaaS.pem")

engine = create_engine(url, connect_args={"sslcontext": sslcontext})

In [ ]:
db = IRISVector.from_documents(
    embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"),
    documents = docs,
    collection_name = 'state_of_the_union',
    connection_string = url,
    engine_args = { "connect_args": {"sslcontext": sslcontext} }
)

In [ ]:
print(f"Number of docs in vector store: {len(db.get()['ids'])}")

In [ ]:
query = "Joint patrols to catch traffickers"
docs_with_score = db.similarity_search_with_score(query)

for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)